In [3]:
#MAPREDUCE
"Каково среднее время игры (в часах) у пользователей, которые написали отзыв во время раннего доступа (Early Access)?"

import pandas as pd

def map_chunk(df_chunk):
    total_playtime = 0
    count = 0

    for early, playtime in zip(
        df_chunk["written_during_early_access"],
        df_chunk["author.playtime_forever"]
    ):
        if early:
            count += 1
            total_playtime += playtime
    return {"sum": total_playtime, "count": count}


def shuffle(results):
    total_s = 0
    total_c = 0
    for r in results:
        total_s += r["sum"]
        total_c += r["count"]
    return total_s, total_c


def reduce(total_s, total_c):
    if total_c == 0:
        return 0
    return total_s / total_c / 60


def mapReduce(file_path, chunksize=200000):
    mapped = []

    for chunk in pd.read_csv(
        file_path,
        usecols=[
            "written_during_early_access",
            "author.playtime_forever"
        ],
        chunksize=chunksize
    ):
        chunk["written_during_early_access"] = chunk["written_during_early_access"].astype(bool)
        chunk["author.playtime_forever"] = pd.to_numeric(chunk["author.playtime_forever"], errors="coerce").fillna(0)


        mapped.append(map_chunk(chunk))

    total_s, total_c = shuffle(mapped)

    avg_hours = reduce(total_s,total_c)

    return total_c, avg_hours


file_path = "path_to_file.csv"

count, avg = mapReduce(file_path)

print("=== MAPREDUCE ===")
print("Количество пользователей раннего доступа:", count)
print("Среднее время игры (часы):", avg)




=== MAPREDUCE ===
Количество пользователей раннего доступа: 2016385
Среднее время игры (часы): 281.34454032009427
